# Drinkwaterwinning Bleijerheide

## Inladen Python pakketten

In [ ]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

Nu laden we het getrainde model in:

In [ ]:
model = tf.keras.models.load_model('./model')

De test periode is een periode waarvoor het model niet is getrained. 

*(De periode hiervoor volgt nu uit wat er in de onderstaande CSV-bestanden staat, we bouwen nog in dit notebook in dat deze periode flexibel is op te geven.)*

X_test is de afvoer van verhard gebied en dakoppervlak die is voorbewerkt.
Dit is de invoerdata voor het model.

Y_test is de bijbehorende neerslag (geen invoer, maar validatie).

In [ ]:
os.getcwd()

In [ ]:
X_test = pd.read_csv('data/X_test_scaled_new.csv', index_col=0, parse_dates=True)
Y_test = pd.read_csv('data/Y_test_new.csv', index_col=0, parse_dates=True).rename(columns={'P':'P_meting'}, inplace=False)

We stoppen X_test in het model en daarkomt dan een gesimuleerde neerslag uit, y_hat:

In [ ]:
y_hat = pd.DataFrame(index=X_test.index, data=model.predict(X_test), columns=['P_model'])
# soms zijn waarden heel licht 0
y_hat[y_hat['P_model'] < 0.] = 0.

In [ ]:
def plot_resultaat(Y_test, y_hat, path=None):
    Path('figuren_model').mkdir(parents=False, exist_ok=True)
    # Functie om tijdreeksen te plotten
    fig, ax = plt.subplots(1,1,figsize=(14,4))
    ax.plot(y_hat.index, y_hat['P_model'], 'orange', linestyle='-', zorder=1, label='P_model')
    ax.plot(Y_test.index, Y_test['P_meting'], zorder=0, label='P_meting')
    plt.xlabel('Datum')
    plt.ylabel('Neerslag [mm/]')
    plt.legend()
    plt.grid(True)
    ax.set_xlim(y_hat.index[0], y_hat.index[-1])
    plt.gca().spines['top'].set_visible(False)
    plt.gca().spines['right'].set_visible(False)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(r'figuren_model/tijdreeksen_model_testperiode.png')

We plotten zowel de gemeten als de voorspelde neerslag...

In [ ]:
plot_resultaat(Y_test, y_hat)

... en schrijven ook het resultaat naar csv.

In [ ]:
output = pd.concat([y_hat, Y_test], axis=1, ignore_index=False)
Path('tabellen_model').mkdir(parents=False, exist_ok=True)
output.to_csv(r'tabellen_model/model_testperiode.csv', sep=",")